In [7]:
import os
from pathlib import Path
# Check if the code is running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    base_path = "/content/"
    if Path(f"{base_path}final_project").is_dir():
      %cd {base_path}final_project
      !git pull
      %cd {base_path}
    else:
      !git clone https://github.com/fernandaluft/final_project.git
except ImportError:
    IN_COLAB = False
    base_path = "/workspaces/"

/content/final_project
Already up to date.
/content


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from final_project.src.scraping import Scraping

In [9]:
if IN_COLAB==True:
  scraping = Scraping(IN_COLAB)
  scraping.kaggle_scrape()

In [10]:
limit = 5000

In [11]:
class Eda():
    def __init__(self, limit):
        if limit != None:
            self.df1 = pd.read_csv(f'{base_path}final_project/data/books_data.csv').sample(limit)
            self.df2 = pd.read_csv(f'{base_path}final_project/data/Books_rating.csv').sample(limit)
        else:
            self.df1 = pd.read_csv(f'{base_path}final_project/data/books_data.csv')
            self.df2 = pd.read_csv(f'{base_path}final_project/data/Books_rating.csv')

    def merge_datasets(self):
        self.books_df = self.df1.merge(self.df2, left_on='Title', right_on='Title', how='inner')
        os.makedirs(f'{base_path}final_project/preprocessed_data', exist_ok=True)
        self.books_df.to_csv(f"{base_path}final_project/preprocessed_data/books_reviews.csv", index=False)
        return self.books_df

    def print_info(self):
        print(self.books_df.info())
        print(self.books_df.head())

In [12]:
eda = Eda(limit)
books_df = eda.merge_datasets()
eda.print_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 0 to 118
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               118 non-null    object 
 1   description         93 non-null     object 
 2   authors             108 non-null    object 
 3   image               108 non-null    object 
 4   previewLink         111 non-null    object 
 5   publisher           94 non-null     object 
 6   publishedDate       111 non-null    object 
 7   infoLink            111 non-null    object 
 8   categories          100 non-null    object 
 9   ratingsCount        65 non-null     float64
 10  Id                  119 non-null    object 
 11  Price               18 non-null     float64
 12  User_id             96 non-null     object 
 13  profileName         96 non-null     object 
 14  review/helpfulness  119 non-null    object 
 15  review/score        119 non-null    float64
 16  review/t

In [13]:
%cd {base_path}final_project
#!git config --global user.name "fernandaluft"
#!git config --global user.email "fernandaluft@gmail.com"
!git config --global user.name "neural1977"
!git config --global user.email "pugliese05@gmail.com"
!git add .
!git commit -m "New Final Project"


/content/final_project
[main 707a513] New Final Project
 1 file changed, 120 insertions(+)
 create mode 100644 preprocessed_data/books_reviews.csv


In [14]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address
